# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Libraries</a></div><div class="lev1 toc-item"><a href="#Define-Arguments" data-toc-modified-id="Define-Arguments-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define Arguments</a></div><div class="lev1 toc-item"><a href="#Function" data-toc-modified-id="Function-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Function</a></div>

# Import Libraries

In [5]:
"""
addm_test.py
Author: Gabriela Tavares, gtavares@caltech.edu

Test to check the validity of the addm parameter estimation. Artificil data is
generated using specific parameters for the model. Fixations are sampled from
the data pooled from all subjects (or from a subset of subjects, when
provided). The parameters used for data generation are then recovered through a
posterior distribution estimation procedure.
"""

import argparse
import numpy as np

from addm import aDDM
from util import (load_data_from_csv, get_empirical_distributions,
                  convert_item_values)

# Define Arguments

In [6]:
class arguments:
    
    # "--subject-ids", nargs="+", type=str, default=[],
    # help= List of subject ids. If not provided, all
    # existing subjects will be used.")
    subject_ids = []
        
    # "--num-threads", type=int, default=9,
    # help= Size of the thread pool."
    num_threads = 4
    
    # --trials-per-condition", type=int, default=800,
    # help= Number of artificial data trials to be
    # generated per trial condition."
    trials_per_condition = 100
    
    # "--d", type=float, default=0.006,
    # help= aDDM parameter for generating artificial data.
    d = 0.006
    
    # "--sigma", type=float, default=0.08,
    # help= aDDM parameter for generating artificial data.
    sigma = 0.08
    
    # "--theta", type=float, default=0.5,
    # help= aDDM parameter for generating artificial data.
    theta = 0.5
    
    # "--range-d", nargs="+", type=float, default=[0.005, 0.006, 0.007],
    # help= Search range for parameter d.
    range_d = [0.005, 0.006, 0.007]
    
    # "--range-sigma", nargs="+", type=float, default=[0.065, 0.08, 0.095],
    # help= Search range for parameter sigma.
    range_sigma = [0.065, 0.08, 0.095]
    
    # "--range-theta", nargs="+", type=float, default=[0.4, 0.5, 0.6],
    # help= Search range for parameter theta.
    range_theta = [0.4, 0.5, 0.6]
    
    # "--expdata-file-name", type=str, default="expdata.csv",
    # help= Name of experimental data file.
    expdata_file_name = "expdata.csv"
    
    # "--fixations-file-name", type=str, default="fixations.csv",
    # help= Name of fixations file.
    fixations_file_name = "fixations.csv"
    
    # "--verbose", default=False, action="store_true",
    # help= Increase output verbosity.
    verbose = "store_true"
        
args = arguments()

# Function

In [ ]:
def main():

    # Trial conditions with format (valueLeft, valueRight). Change this
    # according to the experiment.
    trialConditions = [(0, 0), (0, 1), (0, 1), (0, 2), (0, 2), (0, 3),
                       (1, 0), (1, 0), (1, 1), (1, 2), (1, 2), (1, 3),
                       (2, 0), (2, 0), (2, 1), (2, 1), (2, 2), (2, 3),
                       (3, 0), (3, 1), (3, 2)
                      ]

    # Load experimental data from CSV file.
    if args.verbose:
        print("Loading experimental data...")
    data = load_data_from_csv(
        args.expdata_file_name, args.fixations_file_name,
        convertItemValues=convert_item_values)

    # Get fixation distributions.
    if args.verbose:
        print("Getting fixation distributions...")
    subjectIds = args.subject_ids if args.subject_ids else None
    fixationData = get_empirical_distributions(data, subjectIds=subjectIds)

    # Generate artificial data.
    if args.verbose:
        print("Generating artificial data...")
    model = aDDM(args.d, args.sigma, args.theta)
    trials = list()
    for (valueLeft, valueRight) in trialConditions:
        for t in xrange(args.trials_per_condition):
            try:
                trials.append(
                    model.simulate_trial(valueLeft, valueRight, fixationData))
            except:
                print("An exception occurred while generating artificial " +
                      "trial " + str(t) + " for condition (" + str(valueLeft) +
                      ", " + str(valueRight) + ").")
                raise

    # Get likelihoods for all models and all artificial trials.
    numModels = (len(args.range_d) * len(args.range_sigma) *
                 len(args.range_theta))
    likelihoods = dict()
    models = list()
    posteriors = dict()
    for d in args.range_d:
        for sigma in args.range_sigma:
            for theta in args.range_theta:
                model = aDDM(d, sigma, theta)
                if args.verbose:
                    print("Computing likelihoods for model " +
                          str(model.params) + "...")
                try:
                    likelihoods[model.params] = model.parallel_get_likelihoods(
                        trials, numThreads=args.num_threads)
                except:
                    print("An exception occurred during the likelihood " +
                          "computations for model " + str(model.params) + ".")
                    raise
                models.append(model)
                posteriors[model.params] = 1. / numModels

    # Compute the posteriors.
    for t in xrange(len(trials)):
        # Get the denominator for normalizing the posteriors.
        denominator = 0
        for model in models:
            denominator += (posteriors[model.params] *
                            likelihoods[model.params][t])
        if denominator == 0:
            continue

        # Calculate the posteriors after this trial.
        for model in models:
            prior = posteriors[model.params]
            posteriors[model.params] = (likelihoods[model.params][t] *
                                        prior / denominator)

    if args.verbose:
        for model in models:
            print("P" + str(model.params) +  " = " +
                  str(posteriors[model.params]))
        print("Sum: " + str(sum(posteriors.values())))


if __name__ == "__main__":
    main()

Loading experimental data...
Getting fixation distributions...
Generating artificial data...
Computing likelihoods for model (0.005, 0.065, 0.4)...
Computing likelihoods for model (0.005, 0.065, 0.5)...
Computing likelihoods for model (0.005, 0.065, 0.6)...
Computing likelihoods for model (0.005, 0.08, 0.4)...
Computing likelihoods for model (0.005, 0.08, 0.5)...
Computing likelihoods for model (0.005, 0.08, 0.6)...
Computing likelihoods for model (0.005, 0.095, 0.4)...
Computing likelihoods for model (0.005, 0.095, 0.5)...
Computing likelihoods for model (0.005, 0.095, 0.6)...
Computing likelihoods for model (0.006, 0.065, 0.4)...
Computing likelihoods for model (0.006, 0.065, 0.5)...
Computing likelihoods for model (0.006, 0.065, 0.6)...
Computing likelihoods for model (0.006, 0.08, 0.4)...
Computing likelihoods for model (0.006, 0.08, 0.5)...
Computing likelihoods for model (0.006, 0.08, 0.6)...
Computing likelihoods for model (0.006, 0.095, 0.4)...
Computing likelihoods for model (

In [8]:
aDDM

addm.aDDM

NameError: name 'likelihoods' is not defined